In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,John Prine,Dear John,(vocals by john prine) \r\n \r\nWhen I woke ...
1,Aled Jones,All Through The NIght,"Holl amrantau'r ser ddywedant, ar hyd y nos \..."
2,Tom Waits,New Coat Of Paint,Let's put a new coat of paint \r\nOn this lon...
3,Bob Marley,Is This Love,"I want to love you, and treat you right, \r\n..."
4,System Of A Down,The Metro,"I'm alone, sitting with my broken glass \r\nM..."
5,Carly Simon,No Secrets,(carly simon) \r\n \r\nWe have no secrets \...
6,Natalie Imbruglia,Perfectly,When I say it doesn't matter \r\nIt matters m...
7,Michael W. Smith,She Walks With Me,From the first breath of her life \r\nShe fle...
8,Pearl Jam,Moline,[Verse 1:] \r\nI knew there was trouble \r\n...
9,Sam Smith,I've Told You Now,You know what I mean \r\nIt's like walking in...


In [9]:
df['text'][0]

'(vocals by john prine)  \r\n  \r\nWhen I woke up this morning  \r\nThere was a note upon my door  \r\nSaying " don\'t make me no coffee, babe  \r\nCause I won\'t be back no more"  \r\n  \r\nAnd that\'s all she wrote  \r\n"dear john", I sent your saddle home"  \r\n  \r\nNow jonah got along in the belly of a whale  \r\nDaniel in the lion\'s den  \r\nI know a guy that didn\'t try to get along  \r\nAnd he won\'t get a chance again  \r\n  \r\nThat\'s all she wrote  \r\n"dear john", I sent your saddle home"  \r\n  \r\nShe didn\'t forward no address  \r\nNo she never said goodbye  \r\nAll she said was "if you get blue  \r\nJust hang your little head and cry"  \r\n  \r\nThat\'s all she wrote  \r\n"dear john", I sent your saddle home"  \r\n  \r\nNow my gal\'s short and stubby  \r\nShe\'s mean as she can be  \r\nIf that little old gal of mine  \r\nEver gets a hold of me  \r\n  \r\nThat\'s all she wrote  \r\n"dear john", I fetched your saddle home"  \r\n  \r\nWent down to the bank this morning  

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.        , 0.01068763, ..., 0.01486303, 0.01601696,
       0.02608799])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))